In [1]:
import os
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import six.moves.cPickle
from keras.utils.np_utils import to_categorical
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

negatives = {
    "didn't": "didn_`_t",
    "couldn't": "couldn_`_t",
    "don't": "don_`_t",
    "wouldn't": "wouldn_`_t",
    "doesn't": "doesn_`_t",
    "wasn't": "wasn_`_t",
    "weren't": "weren_`_t",
    "shouldn't":"shouldn_`_t",
    "isn't": "isn_`_t",
    "aren't": "aren_`_t",
}

MODEL_NAME = "models/model_amazon_2017-03-13-02-14base.hdf5"
TOKENIZER = "amazon_2017-03-13-02-14_tokenizer"
MAX_SEQUENCE_LENGTH = 110

tokenizer = six.moves.cPickle.load(open(TOKENIZER, "rb"))
model = load_model(MODEL_NAME)

Using TensorFlow backend.


In [2]:
def preprocess(text):
    text = text.lower()
    text = text.replace('<br />', ' ')
    text = ' '.join(tweet_tokenizer.tokenize(text))
    for k, v in negatives.items():
        text = text.replace(k, v)
    return text

In [4]:
TEXT_DATA_DIR = ''
TEXT_DATA_FILE = 'test.csv'
HEADER = True

def load_data():
    x = []
    y = []
    with open(os.path.join(TEXT_DATA_DIR, TEXT_DATA_FILE), "r", encoding="utf-8") as f:
        if HEADER:
            _ = next(f)
        for line in f:
            temp_y, temp_x = line.rstrip("\n").split("|", 1)
            x.append(preprocess(temp_x))
            y.append(temp_y)

    return x, y
data, y = load_data()

In [8]:
sequences = tokenizer.texts_to_sequences(data)
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y = to_categorical(np.asarray(y))
score = model.evaluate(X, y, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

149624/149624 [==============================] - 404s   
acc: 95.96%


In [9]:
np.mean(y)

0.5

In [11]:
len(X), len(y)

(149624, 149624)

In [15]:
np.mean(y[:,1])

0.86190049724643103